# LDA(Latent Dirichlet Allocation): 잠재 디리클레 할당

https://wikidocs.net/40710 참고

In [1]:
import pymysql

DB_HOST = 'localhost'
DB_USER = 'pyuser'
DB_PASSWD = 'pyuser'
DB_NAME = 'pyuser'

conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                   db=DB_NAME, charset='utf8')
curs = conn.cursor()
sql = """
    select blog_content from lotte_blog
    where blog_date between '20200101' and '20201231'
"""
curs.execute(sql)
table_data = curs.fetchall() 
conn.close()

In [2]:
import pandas as pd

# df = pd.DataFrame(table_data,columns=['no','url','Date','Title','subtitle','Contents'])
df = pd.DataFrame(table_data,columns=['Contents'])


In [3]:

# df [ 'Contents'   ]  <= Series타입,   df[ ['Contents'] ]  <== dataframe타입
sentences = df[ ['Contents'] ]   

# print로 type(text) 의 결과는 Dataframe타입이어야 함
print(type(sentences))
# <class 'pandas.core.frame.DataFrame'> 이 아니면 오류임

<class 'pandas.core.frame.DataFrame'>


In [4]:
sentences.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15905 entries, 0 to 15904
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Contents  15904 non-null  object
dtypes: object(1)
memory usage: 124.4+ KB


라이브러리 추가 및 클래스 선언

In [5]:
# konlpy 설치 명령어
# !pip install konlpy

In [6]:
from konlpy.tag import Okt
okt=Okt()

명사추출: nouns 명사추출. phrases 어절 추출

In [7]:
sentences['Contents'] = sentences['Contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20008\2540441724.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  sentences['Contents'] = sentences['Contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [8]:
# Null 갯수 확인
print(sentences.isnull().sum())

Contents    1
dtype: int64


In [9]:
sentences.loc[sentences.Contents.isnull()]   # Contents 는 sentences['Contents'] 여기 있는 이름임

,Contents
13836,None


In [10]:
# Null 값을 가진 샘플을 제거
sentences = sentences.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(sentences.isnull().values.any()) # Null 값이 존재하는지 다시 확인. False가 나오면 정상임

False


In [11]:
sentences

,Contents
0,신도림 역에서 디큐브아트센터 롯데시네마 가는길 찍어봤어요 약간 복잡하다 느끼시는 ...
1,오늘은 저의 롯데시네마 선정 소식을 가져왔어요 ㅑ 저는 작년까지 약년간 ㄱㅖ...
2,천호 롯데시네마 맛집 잊을만 하면 생각나는 양꼬치 제가 자주 가는 천호 롯데시네마 ...
3,푸하핫 용인스팀세차용인실내세차두꺼비스팀세차역북롯데시네마용인스팀세차잘하는곳 태어날때부...
4,겨울왕국 자막 롯데시네마 월드타워 수퍼 관 관람기 회차 어제 진짜 오랜만에 ...
...,...
15900,칠곡안경칠곡렌즈칠곡아큐브동천동안경동천동렌즈동천동아큐브비비드대구안경대구안경싼곳칠곡블루...
15901,향남읍사무소 홈플러스향남점롯데시네마 향남버스터미널 화성중앙병원 등 주요시설 분이내 ...
15902,치과병원에서 년간 수련받았고 따라서 연세교정과 전문의회 정식회원으로 등록하였습니다 ...
15903,생활이 편리 아트몰링쇼핑몰롯데시네마중랑천 등 다양하고 쾌적한 인프라 수익을 바로 장...


In [12]:
tokenized_doc = sentences.apply(lambda row: okt.nouns(row['Contents']), axis=1)


단어 길이가 1이하인 단어들은 제거 (길이가 짧은 단어 ex. 은/는/이/가/을/를 등 제거)


In [13]:
tokenized_doc = tokenized_doc.apply(lambda x: [word for word in x if len(word) > 1])


분석에서 제거할 불필요한 단어들 기재

In [14]:
from nltk.corpus import stopwords

stop = ['연합뉴스','무단']
tokenized_doc = tokenized_doc.apply(lambda x: [word for word in x if word not in (stop)])


In [15]:
# 저장된 단어들 확인하기 
print(tokenized_doc)

0        [신도림, 큐브, 아트, 센터, 롯데, 시네마, 약간, 신도림역, 출구, 왼쪽, 보...
1        [오늘, 롯데, 시네마, 선정, 소식, 작년, 소듕, 선정, 뚠뚠, 롯데, 시네마,...
2        [롯데, 시네마, 맛집, 꼬치, 자주, 롯데, 시네마, 맛집, 거리, 꼬치, 영화관...
3        [용인, 스팀, 차용, 실내, 세차, 두꺼비, 스팀, 세차, 역북, 롯데, 시네마,...
4        [겨울왕국, 자막, 롯데, 시네마, 월드, 타워, 수퍼, 관람, 회차, 어제, 진짜...
                               ...                        
15900    [칠곡, 안경, 칠곡, 렌즈, 칠곡, 큐브, 동천동, 안경, 동천동, 렌즈, 동천동...
15901    [향남읍, 사무소, 홈플러스, 향남, 롯데, 시네마, 향남, 버스, 터미널, 화성,...
15902    [치과, 병원, 수련, 따라서, 교정, 전문, 의회, 정식, 회원, 등록, 또한, ...
15903    [생활, 편리, 아트, 몰링, 쇼핑몰, 롯데, 시네마, 중랑천, 인프라, 수익, 바...
15904    [일대, 롯데, 시네마, 병원, 공원, 하천, 생활, 편의, 시설, 위치, 생활, ...
Length: 15904, dtype: object


TF-IDF(Term Frequency - Inverse Document Frequency)는 정보 검색과 텍스트 마이닝에서 이용하는 가중치로, 
여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치

https://wikidocs.net/31698 참고

TF-IDF의 TfidfVectorizer는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용함.
이를 사용하기 위해 다시 토큰화 작업을 역으로 취소하는 역토큰화(Detokenization)작업을 수행

# gensim을 통해서 LDA를 수행하고, 시각화

https://wikidocs.net/30708

In [16]:
from gensim import corpora

dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

In [17]:
import gensim

NUM_TOPICS = 10 # n개의 토픽, k=n

# passes는 알고리즘의 동작 횟수
# 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 설정함
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)   # TOPIC별 단어를 4개만 보여주기, 숫자를 늘리면 더 많은 단어들이 보여짐
for topic in topics:
    print(topic)


(0, '0.062*"할인" + 0.041*"롯데" + 0.038*"시네마" + 0.020*"천원"')
(1, '0.051*"시네마" + 0.047*"롯데" + 0.045*"건물" + 0.037*"주차"')
(2, '0.024*"롯데" + 0.020*"시네마" + 0.019*"병점" + 0.018*"개봉"')
(3, '0.073*"부평" + 0.032*"인천" + 0.032*"원주" + 0.028*"단구동"')
(4, '0.057*"대구" + 0.037*"시네마" + 0.036*"롯데" + 0.035*"카페"')
(5, '0.050*"롯데" + 0.030*"시네마" + 0.027*"시설" + 0.020*"생활"')
(6, '0.054*"시네마" + 0.049*"롯데" + 0.045*"맛집" + 0.028*"위치"')
(7, '0.084*"롯데" + 0.055*"시네마" + 0.008*"백화점" + 0.008*"바로"')
(8, '0.089*"롯데" + 0.083*"시네마" + 0.063*"영화" + 0.016*"영화관"')
(9, '0.045*"시네마" + 0.040*"롯데" + 0.037*"상가" + 0.021*"입점"')


각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줌
또한 맨 앞에 있는 토픽 번호는 0부터 시작하므로 총 N개의 토픽은 0부터 N-1까지의 번호가 할당되어져 있음

위의 출력 결과에서 (숫자, 확률)은 각각 토픽 번호와 해당 토픽이 해당 문서에서 차지하는 분포도를 의미함
예를 들어 0번째 문서의 토픽 비율에서 (0, '0.023*"경제..."'')은 0번 토픽에서 경제가 2.3%의 분포도를 가지는 것을 의미함

In [18]:
# !pip install pyLDAvis

In [19]:

import pyLDAvis.gensim_models
import pyLDAvis.gensim_models as gensimvis


pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)

# https://lovit.github.io/nlp/2018/09/27/pyldavis_lda/
# 시각화는 1- 부터 시작함
pyLDAvis.display(vis)

C:\Users\Eddie\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


λ라는 값은 [0, 1] 사이에서 조절가능함
λ를 1 로 설정하면 토픽 별로 가장 자주 등장하는 단어들을 우선적으로 키워드로 선택한다는 의미이며,
λ를 0 에 가깝게 설정할수록 토픽 간에 차이가 많이 나는 단어를 선택한다는 의미임